In [14]:
from urllib.request import urlopen  # b_soup_1.py
from bs4 import BeautifulSoup
import csv
import re

Must run the following commands in the appropriate conda env in order for this to work:
>  \> conda install spacy  
>  \> python -m spacy download en   
 (second one must be run as admin)

Alternatively run the top command and then replace the cell below with:  
`import spacy  
import en_core_web_sm  
nlp = en_core_web_sm.load()`  

In [2]:
import spacy
nlp = spacy.load('en')

In [3]:
# driver
def get_skill_map(skills_list):
    """
    Returns a dict mapping each skill in the skill_list
    to a list of course names associated with that skill.
    """
    # first get the links for all Heinz courses
    course_dict = get_course_links()
    course_descriptions = course_dict.copy()

    # then scrape the description text for each link
    for name, link in course_dict.items():
        course_descriptions[name] = get_course_description(link)
        
    # time for a little nlp (to lemmatize the descriptions)
    # we'll store the results in a new dictionary
    parsed_descriptions = course_descriptions.copy()
    for name, text in course_descriptions.items():
        parsed_text = nlp(text)
        parsed_tokens = []
        
        # remove all stopwords, punctuation, and spaces
        for token in parsed_text:
            lemma = token.lemma_.lower()
            if not (nlp.vocab[lemma].is_stop or token.pos_ == 'PUNCT' or token.pos_ == 'SPACE'):
                parsed_tokens.append(lemma)
        
        # finally add list to dictionary
        parsed_descriptions[name] = parsed_tokens
    
    
    # now we can iterate through the skills list and build our final dictionary
    # give each skill its own list
    skills_to_courses = { k : [] for k in skill_list }
    
    for skill in skills_list:
        # now check every course for that skill
        for course_name in parsed_descriptions.keys():
            # if that skill appears in the description, add to the map
            if skill.lower() in parsed_descriptions[course_name]:
                skills_to_courses[skill].append(course_name)
    
    # return the final mapping
    return skills_to_courses

In [4]:
def get_course_links():
    """
    Get a list of urls to Heinz College course description 
    pages, so we can scrape each of those iteratively.
    
    Params
    -------
    skill_list
        A list of all the skills searched for in these pages
    
    Return
    -------
    <dict> {course_name: course_link, ...}
        course_name: string containing course name
        course_link: string containing link to course description page
    """
    html_base = 'https://api.heinz.cmu.edu'
    html = urlopen('https://api.heinz.cmu.edu/courses_api/course_list/')
    soup = BeautifulSoup(html.read(), "lxml")
    
    names = []
    links = []
    
    # Each course link is contained in a <tr> element with class="clickable-row"
    course_rows = soup.findAll('tr', {'class': 'clickable-row'})
    for row in course_rows:
        names.append(row.find_all('td')[1].string)
        links.append(html_base + row.get('data-href'))
    
    return dict(zip(names, links))

In [11]:
def get_course_description(link):
    # Retrieve the html
    html = urlopen(link)
    soup = BeautifulSoup(html.read(), "lxml")

    # Recover the description text
    text = ' '.join([p.get_text() for p in soup.find_all('p')])
    clean = re.sub(r'[^\w\s]', '', text)
    return clean

In [9]:
html = urlopen('https://api.heinz.cmu.edu/courses_api/course_list/')
soup = BeautifulSoup(html.read(), "lxml")

In [10]:
course_rows = bsyc.findAll('tr', {'class': 'clickable-row'})

In [7]:
course_links = get_course_links()
course_links['Statistical Reasoning']

'https://api.heinz.cmu.edu/courses_api/course_detail/90-707'

In [12]:
html = urlopen(course_links['Statistical Reasoning'])
soup = BeautifulSoup(html.read(), "lxml")

In [15]:
t = get_course_description(course_links['Statistical Reasoning'])
t

'Statistical Reasoning Units 12 Description This course will provide an introduction to the principles of data collection description and analysis You will learn the basic tools of statistical inference and modeling as well as how to interpret results measure the uncertainty in result and understand the limitations of results You will also learn how to interpret statistical output and how not to be fooled by statistical studies Learning Outcomes At the end of this course you should be able to\r\n Present data visually in tabular and graphic form\r\n Summarize a set of observations by reporting a measure of center and dispersion\r\n Explain how and why sample data can be used to estimate descriptive measures of populations when census data is unavailable and how we measure the accuracy and precision of the estimate\r\n Apply the basic rules of probability\r\n Find and interpret the probability for a random variable which has a normal distribution\r\n Explain how to take a proper scienti

In [10]:
text = ' '.join([p.get_text() for p in soup.find_all('p')])
text

'Statistical Reasoning Units: 12 Description: This course will provide an introduction to the principles of data collection, description and analysis. You will learn the basic tools of statistical inference and modeling, as well as how to interpret results, measure the uncertainty in result, and understand the limitations of results. You will also learn how to interpret statistical output, and how not to be fooled by statistical studies. Learning Outcomes: At the end of this course, you should be able to\r\n- Present data visually in tabular and graphic form\r\n- Summarize a set of observations by reporting a measure of center and dispersion\r\n- Explain how and why sample data can be used to estimate descriptive measures of populations when census data is unavailable, and how we measure the accuracy and precision of the estimate\r\n- Apply the basic rules of probability\r\n- Find and interpret the probability for a random variable which has a normal distribution\r\n- Explain how to ta

In [ ]:
# test it
test_list = ['Management', 'software', 'knowledge']
results = get_skill_map(test_list)
write_to_csv(results)